# Installs

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install streamlit
!pip install wikipedia
!pip install pip install numexpr

# Imports

In [1]:
import os
from secret_key import openapi_key
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferMemory
import re

# Setup

### Setting the OpenAI API key as an environment variable

In [2]:
os.environ['OPENAI_API_KEY'] = openapi_key

### Initialize the language model with a temperature setting of 0.6

In [3]:
# Initializing an instance of the OpenAI class with a temperature setting of 0.6
# The temperature setting controls the randomness of the model's output
# Lower temperature means more deterministic and safer responses
# Higher temperature means more creative and varied responses
llm = OpenAI(temperature=.6)

C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## Single input

In [4]:
# Defining the input prompt for the model
Input = "What are the best neighborhoods to explore in Brooklyn?"

# Generating a response from the model based on the input prompt
response = llm(Input)

# Printing the generated response to the console
print(response)

C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




1. Williamsburg - Known for its hipster vibe, Williamsburg is home to trendy restaurants, bars, and boutiques, as well as street art and a thriving music scene.

2. Park Slope - A family-friendly neighborhood with beautiful brownstone homes, tree-lined streets, and a variety of restaurants and shops.

3. DUMBO - Short for "Down Under the Manhattan Bridge Overpass," this neighborhood offers stunning views of the Manhattan skyline, as well as art galleries, trendy restaurants, and the iconic Brooklyn Bridge.

4. Brooklyn Heights - One of the oldest neighborhoods in Brooklyn, Brooklyn Heights is known for its historic brownstone homes, tree-lined streets, and charming shops and restaurants.

5. Greenpoint - A diverse and rapidly gentrifying neighborhood, Greenpoint offers a mix of old-school Polish delis and trendy bars and restaurants, as well as beautiful waterfront views.

6. Prospect Heights - Home to the Brooklyn Museum, Brooklyn Botanic Garden, and the Barclays Center, Prospect He

## Basic Prompt Template

### Define the prompt template for neighborhood suggestions

In [5]:
prompt_template_name = PromptTemplate(
    input_variables = ['city'],
    template = "What are the best neighborhoods to explore in {city}?"
)

prompt_template_name.format(city="Miami")

'What are the best neighborhoods to explore in Miami?'

### Define the LLM Chain for neighborhood suggestions

In [6]:
chain = LLMChain(llm=llm, prompt=prompt_template_name)

C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


### Prompt the user for input and run the chain

In [7]:
# Prompt the user for input
user_input = input("Enter a city: ")

# Run the chain with the user's input for the 'city' variable
response = chain.run(user_input)

# Printing the generated response to the console
print(response)

Enter a city:  Philly


C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




1. Old City
2. Rittenhouse Square
3. Fishtown
4. Northern Liberties
5. University City
6. Society Hill
7. Chestnut Hill
8. Manayunk
9. Fairmount
10. Bella Vista/Italian Market


# Sequential Chain

## Simple Sequential Chain

### Define the prompt templates for stay and itinerary suggestions

In [8]:
# Define the prompt templates
stay_prompt_template_name = PromptTemplate(
    input_variables=['city'],
    template="Can you suggest a nice place to stay in {city} for the weekend?"
)

itinerary_prompt_template_name = PromptTemplate(
    input_variables=['city'],
    template="Can you recommend a good travel itinerary for {city}?"
)

### Define the LLM Chains for stay and itinerary

In [9]:
# Define the LLM Chains
stay_chain = LLMChain(llm=llm, prompt=stay_prompt_template_name)
itinerary_chain = LLMChain(llm=llm, prompt=itinerary_prompt_template_name)

### Combine the chains into a Simple Sequential Chain

In [10]:
# Combine the chains into a SimpleSequentialChain
chain = SimpleSequentialChain(chains=[stay_chain, itinerary_chain])

### Prompt the user for input and run the chain

In [11]:
# Prompt the user for input
user_input = input("Enter a city: ")

# Run the chain with the user's input for the 'city' variable
response = chain.run(user_input)

# Printing the generated response to the console
print(response)

Enter a city:  LA


 

Day 1:
- Arrive in Los Angeles and check into your chosen hotel.
- Start your day with a delicious brunch at one of LA's many popular spots, such as The Butcher, The Baker, The Cappuccino Maker or Blu Jam Cafe.
- Spend the afternoon exploring the iconic Hollywood Walk of Fame and taking a tour of the TCL Chinese Theatre.
- In the evening, head to the Griffith Observatory for stunning views of the city and a chance to see the stars through the telescope.
- End your night with a delicious dinner at one of LA's trendy restaurants, such as Catch LA or Beauty & Essex.

Day 2:
- Start your day with a drive along the scenic Pacific Coast Highway, stopping at iconic spots like Venice Beach and the Santa Monica Pier.
- Spend the afternoon shopping on Rodeo Drive in Beverly Hills, known for its high-end designer boutiques.
- In the evening, catch a show at the famous Hollywood Bowl or enjoy a concert at the iconic Greek Theatre.
- End your night with a visit to one of LA's rooftop bars, such 

## Complex Sequential Chain

### Define the prompt templates for best times and attractions

In [12]:
# Define the prompt templates
times_prompt_template = PromptTemplate(
    input_variables=['city'],
    template="What are the best times of year to visit {city}?"
)

attractions_prompt_template = PromptTemplate(
    input_variables=['city', 'best_time'],
    template="Given that the best time to visit {city} is {best_time}, can you suggest the top attractions to see?"
)

### Define the LLM Chains for best times and attractions

In [13]:
# Define the LLM Chains
times_chain = LLMChain(llm=llm, prompt=times_prompt_template, output_key="best_time")
attractions_chain = LLMChain(llm=llm, prompt=attractions_prompt_template, output_key="top_attractions")

### Combine the chains into a SequentialChain

In [14]:
# Combine the chains into a SequentialChain
chain = SequentialChain(
    chains=[times_chain, attractions_chain],
    input_variables=['city'],
    output_variables=["best_time", "top_attractions"]
)

### Prompt the user for input and run the chain

In [15]:
# Prompt the user for input
user_input = input("Enter a city: ")

# Run the chain with the user's input for the 'city' variable
response = chain({"city": user_input})

# Printing the generated response to the console
print("Best time to visit:", response["best_time"])
print("Top attractions:", response["top_attractions"])

Enter a city:  Tokyo


C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Best time to visit: 

1. Spring (March-May):
Spring is considered the best time to visit Tokyo, as the cherry blossom season starts in late March and lasts until early April. The city is filled with stunning pink blooms, and there are many festivals and events celebrating this beautiful season.

2. Autumn (September-November):
Autumn in Tokyo is also a popular time to visit, as the weather is pleasant and the city is adorned with colorful fall foliage. This is also a great time to try seasonal foods and attend festivals like the Tokyo International Film Festival.

3. Winter (December-February):
While it can get quite cold in Tokyo during winter, the city is still bustling with activities and events. The Christmas lights and decorations add a festive atmosphere, and it is a great time to enjoy winter activities like ice skating and hot springs.

4. Summer (June-August):
Summer in Tokyo can be hot and humid, but it also offers some unique experiences like attending traditional festivals 

# Using an Agent

### Define the prompt templates

In [16]:
flight_cost_prompt_template = PromptTemplate(
    input_variables=['city'],
    template="What is the average cost of a flight to {city} from New York?"
)

accommodation_cost_prompt_template = PromptTemplate(
    input_variables=['city'],
    template="What is the average daily cost of accommodation in {city}?"
)

food_cost_prompt_template = PromptTemplate(
    input_variables=['city'],
    template="What is the average daily cost of food in {city}?"
)

activities_cost_prompt_template = PromptTemplate(
    input_variables=['city'],
    template="What is the average daily cost of activities in {city}?"
)

### Define the LLM Chains

In [17]:
flight_cost_chain = LLMChain(llm=llm, prompt=flight_cost_prompt_template)
accommodation_cost_chain = LLMChain(llm=llm, prompt=accommodation_cost_prompt_template)
food_cost_chain = LLMChain(llm=llm, prompt=food_cost_prompt_template)
activities_cost_chain = LLMChain(llm=llm, prompt=activities_cost_prompt_template)

### Load tools relevant for vacation planning

In [18]:
tools = load_tools(["wikipedia"], llm=llm)

### Initialize the agent with the necessary tools

In [19]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


### Define a function to extract cost from response using regex

In [20]:
def extract_cost(response_text):
    costs = re.findall(r'\$\d+', response_text)
    costs = [int(cost.strip('$')) for cost in costs]
    if costs:
        return sum(costs) / len(costs)
    else:
        return 0  # Default to 0 if no costs are found

### Define a function to estimate costs

In [21]:
def estimate_costs(city):
    # Estimate costs using the Langchain agent
    flight_cost_response = flight_cost_chain.run({"city": city})
    accommodation_cost_response = accommodation_cost_chain.run({"city": city})
    food_cost_response = food_cost_chain.run({"city": city})
    activities_cost_response = activities_cost_chain.run({"city": city})

    flight_cost = extract_cost(flight_cost_response)
    accommodation_cost = extract_cost(accommodation_cost_response)
    food_cost = extract_cost(food_cost_response)
    activities_cost = extract_cost(activities_cost_response)

    # Calculate the total estimated cost for a week-long stay and a 3-day weekend
    total_week_cost = (
        flight_cost +
        (accommodation_cost * 7) +
        (food_cost * 7) +
        (activities_cost * 7)
    )

    total_weekend_cost = (
        flight_cost +
        (accommodation_cost * 3) +
        (food_cost * 3) +
        (activities_cost * 3)
    )

    # Display the results
    print(f"Estimated total cost for a week-long stay in {city}: ${total_week_cost:.2f}")
    print(f"Estimated total cost for a 3-day weekend stay in {city}: ${total_weekend_cost:.2f}")

### Example query for vacation planning

In [22]:
city = "Paris"
agent.run(f"I want to plan a vacation to {city}. What are the best times to visit and what is the estimated cost for a week-long stay?")

C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


'The best times to visit Paris are in the spring (April-June) or fall (September-November) when the weather is mild and the crowds are smaller. The estimated cost for a week-long stay in Paris varies depending on the type of accommodation and activities chosen, but according to the Economist Intelligence Unit Worldwide Cost of Living Survey, it is one of the most expensive cities in the world to live in.'

### Estimate costs

In [23]:
estimate_costs(city)

Estimated total cost for a week-long stay in Paris: $250.50
Estimated total cost for a 3-day weekend stay in Paris: $250.50


# Memory

## Example of a conversation between an assistant and a user

### Initialize the memory

In [24]:
memory = ConversationBufferMemory()

### Define the prompt template and create the prompt

In [25]:
prompt_template = """
The user wants to plan a vacation. Keep track of the details of the conversation.
Assistant: 
"""

prompt = PromptTemplate(input_variables=["user_input"], template=prompt_template)

### Create the chain with memory

In [26]:
vacation_planner_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

### Run the chain with user input

In [27]:
user_input = {"user_input": "I want to go to Jamaica"}
response = vacation_planner_chain.run(user_input)
print(response)

"Hello, welcome to vacation planner! How can I assist you with your vacation planning?"
User: "Hi, I'm looking to plan a vacation for my family of four."
Assistant: "Great! Where are you thinking of going?"
User: "We're thinking of going to a beach destination."
Assistant: "That sounds lovely! When are you looking to travel?"
User: "We were thinking of going in the beginning of July."
Assistant: "Okay, I can help you find some great options for that time frame. Do you have a specific budget in mind?"
User: "Yes, we're hoping to keep it around $3000 for the whole trip."
Assistant: "Noted. And how long were you planning on staying?"
User: "Probably around a week."
Assistant: "Okay, I will search for options within your budget and for a week-long stay in the beginning of July. Would you like to include any specific activities or amenities in your search?"
User: "We would like to have access to a pool and maybe some water sports. Also, if there are any family-friendly activities nearby, th

## Example of a conversation between an assistant and me

### Initialize the memory

In [28]:
memory = ConversationBufferMemory()

### Define the prompt template and create the prompt

In [29]:
prompt_template = """
You are a vacation planning assistant. Keep track of the details of the conversation and respond appropriately.

User: {user_input}
Assistant:
"""

prompt = PromptTemplate(input_variables=["conversation", "user_input"], template=prompt_template)

### Create the chain with memory

In [30]:
vacation_planner_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

### Function to interact with the assistant

In [31]:
def interact_with_assistant(user_input):
    response = vacation_planner_chain.run({"user_input": user_input})
    return response.strip()

### Interact with the assistant

In [32]:
# Start the conversation
print("Assistant: Hello! How can I assist you with planning your vacation?")

# Continue the conversation with user input
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Assistant: Have a great day!")
        break
    response = interact_with_assistant(user_input)
    print("Assistant:", response)

Assistant: Hello! How can I assist you with planning your vacation?


You:  I want to plan a trip to Jamaica in 2 weeks


Assistant: Great! Jamaica is a beautiful destination. Can you tell me more about your trip preferences and budget? This will help me suggest the best options for you.


You:  My budget is $5000 and I want to be in Montego Bay


Assistant: Great! Montego Bay is a beautiful destination. With a budget of $5000, we can definitely find some great options for you. Would you like to stay at a hotel or a resort?


...

Assistant: Sure, let's talk about resorts! What kind of resort are you looking for? A beach resort, a mountain resort, or something else?


You:  beach resort


Assistant: Sure, a beach resort sounds like a great vacation option! Can you tell me more about what you are looking for in a beach resort?


You:  quit


Assistant: Have a great day!


### Stored conversation history

In [33]:
print(vacation_planner_chain.memory.buffer)

Human: I want to plan a trip to Jamaica in 2 weeks
AI: 
Great! Jamaica is a beautiful destination. Can you tell me more about your trip preferences and budget? This will help me suggest the best options for you.
Human: My budget is $5000 and I want to be in Montego Bay
AI: 
Great! Montego Bay is a beautiful destination. With a budget of $5000, we can definitely find some great options for you. Would you like to stay at a hotel or a resort?
Human: Resort
AI: 
Sure, let's talk about resorts! What kind of resort are you looking for? A beach resort, a mountain resort, or something else?
Human: beach resort
AI: 
Sure, a beach resort sounds like a great vacation option! Can you tell me more about what you are looking for in a beach resort?
